In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import sys

In [ ]:
!pip3 install ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
directory = "/content/drive/MyDrive/4.Kurtsoa/BH/BH Praktika/"

sys.path.append(directory)

In [ ]:
from ipynb.fs.full.CDP_Sarrera_Ikasle import sortu_grafoa, bistaratu_grafoa
from ipynb.fs.full.CDP_1_fasea_Ikasle import modularitatea

In [ ]:
if __name__ == "__main__":
  G = sortu_grafoa() #Grafoa sortu
  n = G.number_of_nodes() #n= nodo kopurua
  f = open(directory+"data/ertzak.txt", "w")
  hizt = {}
  edges = list(G.edges)
  #ind, counts = np.unique(edges, return_counts= True)
  



In [ ]:
edge = edges[0][1], edges[0][0]
print(edge in edges)

False


In [ ]:
import random
def generator(G, nodoak, v):
  #nodoak = np.array(list(G.nodes))
  num_nodes = G.number_of_nodes()
  ind = random.sample(range(num_nodes), v)
  aukeratuak = nodoak[ind]
  subGrafo = G.subgraph(list(aukeratuak))
  return subGrafo



In [ ]:
nodoak = np.array(list(G.nodes))
subGraph = generator(G, nodoak, 1300)

In [ ]:
print(subGraph.number_of_nodes())
print(subGraph.number_of_edges())

1300
1570


In [ ]:
irt_pisuak = []
nodoak = list(G.nodes)
for i in range(len(nodoak)):
    pisua = 0 
    for k, v in G[nodoak[i]].items():
        pisua += v["weight"]
    irt_pisuak.append(pisua)

In [ ]:
import random
def generatorv2(G, nodoak, v, irt_pisuak):
  #nodoak = np.array(list(G.nodes))
  prob = np.asarray([G.degree(nodoak[i])+irt_pisuak[i] for i in range(len(nodoak))], dtype = float)
  prob/=np.sum(prob)
  num_nodes = G.number_of_nodes()
  aukeratuak = np.random.choice(range(num_nodes), p = prob, replace = False, size=v)
  subGrafo = G.subgraph(nodoak[aukeratuak])
  return subGrafo

In [ ]:
#Emandako (i,j) indizeek definitzen duten insert eragiketaren hurrengo indizeak itzultzen ditu
def hurrengo_insert(i,j): #i==n denean bukatu behar da
  j_ = j+1
  if j_ == i:
    j_ +=2
  if j_ == i+1:
    j_ +=1
  if j_>n:
    return i+1,0
  return i,j_ 

#Insert inguruneko auzokide bat itzultzen du, emandako (i,j) insert eragiketa kontuan hartuta (i elementua j posizioan txertatu)
#Bestela konbinazio gehiegi (n^2-n), eta RAM memoria xahutzen da.
def insert_auzokide(solution, i=0,j=2):

  neighbour = list(solution)
  neighbour.insert(j,neighbour[i])
  if j>i:
    neighbour.pop(i) #Kendu listatik hasieran zegoen agerpena
  else:
    neighbour.pop(i+1) #Kendu listatik hasieran zegoen agerpena (j<i denez eskuinera mugitu da)
  neighbour = np.asarray(neighbour)

  i,j = hurrengo_insert(i,j)

  return neighbour,i,j

In [ ]:
#Emandako (i,j) indizeek definitzen duten swap eragiketaren hurrengo indizeak itzultzen ditu
def hurrengo_swap(i,j): #i==n denen bukatu behar da
  j_ = j+1
  if j_ == i:
    j_ +=1
  if j_>=n:
    return i+1,i+2
  return i,j_ 

#Swap inguruneko auzokide bat itzultzen du, emandako (i,j) swap eragiketa kontuan hartuta (i eta j posizioetako elementuak trukatu)
def swap_auzokide(solution, i=0,j=1):

  neighbour = np.copy(solution)
  neighbour[i],neighbour[j] = neighbour[j],neighbour[i]

  i,j = hurrengo_swap(i,j)

  return neighbour,i,j

In [ ]:
nodoak = np.array(list(G.nodes))
subGraph = generatorv2(G, nodoak, 1300, irt_pisuak)

In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges())

1843
3215


In [ ]:
print(subGraph.number_of_nodes())
print(subGraph.number_of_edges())

1300
2203


In [ ]:
np.random.choice(range(2),p=[0.4,0.6], replace = False, size=2)

array([0, 1])

In [ ]:
import networkx as nx
from networkx.generators import *
import matplotlib.pyplot as plt

def lortu_zentroa(G):
  #Computing centrality
  degCent = nx.degree_centrality(G)

  #Descending order sorting centrality
  degCent_sorted=dict(sorted(degCent.items(), key=lambda item: item[1],reverse=True))

  #Computing betweeness
  betCent = nx.betweenness_centrality(G, normalized=True, endpoints=True)

  #Descending order sorting betweeness
  betCent_sorted=dict(sorted(betCent.items(), key=lambda item: item[1],reverse=True))

  #Getting indices on top 5 nodes for each measure
  N_top=5
  keys_deg_top=list(degCent_sorted)[0:N_top]
  keys_bet_top=list(betCent_sorted)[0:N_top]

  #Computing centrality and betweeness intersection
  inter_list=list(set(keys_deg_top) & set(keys_bet_top))

  if not inter_list:
    return keys_deg_top[0]
  else:
    return inter_list[0]

In [ ]:
import pickle
import time

nk = [50*i for i in range(1,37)] + [1843]
tamainak = [str(elem) for elem in nk]
lag = ["ins"]*37
izenak = list(map(''.join, zip(lag, tamainak[:-1]))) + ["insOrig"]

for pl in range(len(izenak)):

  n = G.number_of_nodes() 
  nodoak = np.array(list(G.nodes))

  subGr = generatorv2(G, nodoak, nk[pl], irt_pisuak)

  n = subGr.number_of_nodes() #n= nodo kopurua

  zentroa = lortu_zentroa(subGr)

  #Nodoen izenen lista (indizetik izenera pasatzeko)
  nodoak = list(subGr.nodes)

  #Grafoko ertz guztien pisuen batura
  m = subGr.size(weight="weight") 

  #Nodoen izenetatik indizera pasatzeko hiztegia
  indizeak = dict()
  ind = 0
  for nodo in nodoak:
    indizeak[nodo] = ind
    ind+=1

  #Lortu nodo batetik besterako ertzen pisuen matrizea (ertza ez badago pisua=0)
  pisuak = np.zeros((n,n))
  for i in range(n):
      for j in range(n):
        lag = subGr[nodoak[i]]
        if  nodoak[j] in lag: #dira auzokideak
            pisuak[i,j] = lag[nodoak[j]]["weight"]

  #Nodoen graduak eskuratzeko. Dena den, guk graduak erabili ordez nodoari konektatutako ertzen pisuen baturak erabiliko ditugu modularitatean.
  # graduak = []
  # for i in range(n):
  #   graduak.append(G.degree(nodoak[i]))

  #Nodo baten gradua beregana konektatutako ertz guztien pisuen batura kontsideratuko dugu
  graduak = []
  for i in range(n):
    graduak.append(np.sum(pisuak[i]))

  #Modularitate funtziorako aurrekalkulatu i eta j nodo pare posible guztien graduen biderketak 
  graduMul = np.zeros((n,n))
  for i in range(n):
      for j in range(n):
        graduMul[i,j] = graduak[i]*graduak[j]/(2*m)

  #Modularitate funtziorako aurrekalkulatu i eta j nodo pare posible guztien aportazioak edo kontribuzioak
  contributions = (pisuak - graduMul)/(2*m)

  swap_ijs = []
  i,j = 0,1
  while i<n and j<n:
    swap_ijs.append((i,j))
    i,j = hurrengo_swap(i,j)

  insert_ijs = []
  i,j = 0,2
  while i<n:
    insert_ijs.append((i,j))
    i,j = hurrengo_insert(i,j)

  ii, jj = np.ogrid[:n, :n]

  lista = (subGr, nodoak, indizeak, contributions, swap_ijs, insert_ijs, ii, jj, zentroa)

  path = directory + "data/" + izenak[pl]
  outfile = open(path, "wb")
  pickle.dump(lista, outfile)
  outfile.close()

In [ ]:
#ALDAGAI GLOBALAK HASIERATU

#Nodoen izenen lista (indizetik izenera pasatzeko)
nodoak = list(G.nodes)

#Grafoko ertz guztien pisuen batura
m = G.size(weight="weight") 

#Nodoen izenetatik indizera pasatzeko hiztegia
indizeak = dict()
ind = 0
for nodo in nodoak:
  indizeak[nodo] = ind
  ind+=1

#Lortu nodo batetik besterako ertzen pisuen matrizea (ertza ez badago pisua=0)
pisuak = np.zeros((n,n))
for i in range(n):
    for j in range(n):
      lag = G[nodoak[i]]
      if  nodoak[j] in lag: #dira auzokideak
          pisuak[i,j] = lag[nodoak[j]]["weight"]

#Nodoen graduak eskuratzeko. Dena den, guk graduak erabili ordez nodoari konektatutako ertzen pisuen baturak erabiliko ditugu modularitatean.
# graduak = []
# for i in range(n):
#   graduak.append(G.degree(nodoak[i]))

#Nodo baten gradua beregana konektatutako ertz guztien pisuen batura kontsideratuko dugu
graduak = []
for i in range(n):
  graduak.append(np.sum(pisuak[i]))

#Modularitate funtziorako aurrekalkulatu i eta j nodo pare posible guztien graduen biderketak 
graduMul = np.zeros((n,n))
for i in range(n):
    for j in range(n):
      graduMul[i,j] = graduak[i]*graduak[j]/(2*m)

#Modularitate funtziorako aurrekalkulatu i eta j nodo pare posible guztien aportazioak edo kontribuzioak
contributions = (pisuak - graduMul)/(2*m)

swap_ijs = []
i,j = 0,1
while i<n and j<n:
  swap_ijs.append((i,j))
  i,j = hurrengo_swap(i,j)

insert_ijs = []
i,j = 0,2
while i<n:
  insert_ijs.append((i,j))
  i,j = hurrengo_insert(i,j)

Emin = pickle.load(open("/content/drive/MyDrive/4.Kurtsoa/BH/BH Praktika/Emin","rb"))
Emax = pickle.load(open("/content/drive/MyDrive/4.Kurtsoa/BH/BH Praktika/Emax","rb"))